In [1]:
import regex
import pandas as pd

general = "01.01.2017, 01/01/2017, 5€, $5, £5,99, 5,99€, 9,99, 100, U.S.A., AT&T, https://de.dariah.eu"
french = "Français! «Oui», m'éveillai étant Tchouang-tseu... Suis-je réalité? Qui rêve qu'il qui s'imagine!"
spanish = "Español! «Sí», de programación y años después del sueño léxico. Matemático meta-aplicado?"
english = "English! «Yes», it's the 2nd leaf-cutting ant at five o'clock. Didn't I say?"
portuguese = "Português! «Sim», ciência da computação é a ciência que estuda as técnicas. Classifica-se como ciência?"
german = "Deutsch! «Ja», wie geht's? Ein typisches Beispiel für ein System. OK, verbundenes-wort."

In [2]:
french_pattern = regex.compile(r'\p{L}[\p{L}\p{P}]*\p{L}|\p{N}[\p{N}\p{P}]*\p{N}|\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?')
print(french)

Français! «Oui», m'éveillai étant Tchouang-tseu... Suis-je réalité? Qui rêve qu'il qui s'imagine!


In [3]:
pd.Series(french_pattern.findall(general))

0              01.01.2017
1              01/01/2017
2                   £5,99
3                    5,99
4                    9,99
5                     100
6                   U.S.A
7                    AT&T
8    https://de.dariah.eu
dtype: object

In [4]:
pd.Series(french_pattern.findall(french))

0          Français
1               Oui
2        m'éveillai
3             étant
4     Tchouang-tseu
5           Suis-je
6           réalité
7               Qui
8              rêve
9             qu'il
10              qui
11        s'imagine
dtype: object

In [5]:
spanish_pattern = regex.compile(r'\p{N}[\p{N}\p{P}]*\p{N}|\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?|\p{L}[\p{L}\p{P}]*\p{L}|\p{L}{1}')
print(spanish)

Español! «Sí», de programación y años después del sueño léxico. Matemático meta-aplicado?


In [6]:
pd.Series(spanish_pattern.findall(general))

0              01.01.2017
1              01/01/2017
2                   £5,99
3                    5,99
4                    9,99
5                     100
6                   U.S.A
7                    AT&T
8    https://de.dariah.eu
dtype: object

In [7]:
pd.Series(spanish_pattern.findall(spanish))

0           Español
1                Sí
2                de
3      programación
4                 y
5              años
6           después
7               del
8             sueño
9            léxico
10       Matemático
11    meta-aplicado
dtype: object

In [8]:
english_pattern = regex.compile(r'\p{N}[\p{N}\p{P}]*\p{N}|\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?|\p{L}[\p{L}\p{P}]*\p{L}|\p{L}{1}|\p{N}\p{L}+')
print(english)

English! «Yes», it's the 2nd leaf-cutting ant at five o'clock. Didn't I say?


In [9]:
pd.Series(english_pattern.findall(general))

0              01.01.2017
1              01/01/2017
2                   £5,99
3                    5,99
4                    9,99
5                     100
6                   U.S.A
7                    AT&T
8    https://de.dariah.eu
dtype: object

In [10]:
pd.Series(english_pattern.findall(english))

0          English
1              Yes
2             it's
3              the
4              2nd
5     leaf-cutting
6              ant
7               at
8             five
9          o'clock
10          Didn't
11               I
12             say
dtype: object

In [11]:
portuguese_pattern = regex.compile(r'\p{N}[\p{N}\p{P}]*\p{N}|\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?|\p{L}[\p{L}\p{P}]*\p{L}|\p{L}{1}')
print(portuguese)

Português! «Sim», ciência da computação é a ciência que estuda as técnicas. Classifica-se como ciência?


In [12]:
pd.Series(portuguese_pattern.findall(general))

0              01.01.2017
1              01/01/2017
2                   £5,99
3                    5,99
4                    9,99
5                     100
6                   U.S.A
7                    AT&T
8    https://de.dariah.eu
dtype: object

In [13]:
pd.Series(portuguese_pattern.findall(portuguese))

0         Português
1               Sim
2           ciência
3                da
4        computação
5                 é
6                 a
7           ciência
8               que
9            estuda
10               as
11         técnicas
12    Classifica-se
13             como
14          ciência
dtype: object

In [14]:
german_pattern = regex.compile(r'\p{L}[\p{L}\p{P}]*\p{L}|\p{N}[\p{N}\p{P}]*\p{N}|\p{S}?\p{N}[\p{P}\p{N}]{3}\p{S}?')
print(german)

Deutsch! «Ja», wie geht's? Ein typisches Beispiel für ein System. OK, verbundenes-wort.


In [15]:
pd.Series(german_pattern.findall(german))

0              Deutsch
1                   Ja
2                  wie
3               geht's
4                  Ein
5            typisches
6             Beispiel
7                  für
8                  ein
9               System
10                  OK
11    verbundenes-wort
dtype: object